In [3]:
import tkinter as tk
from tkinter import ttk
import xml.etree.ElementTree as ET
import os
import subprocess

def get_automation_folders(master_folder):
    try:
        return [f for f in os.listdir(master_folder) if os.path.isdir(os.path.join(master_folder, f))]
    except FileNotFoundError:
        print(f"Error: Master folder '{master_folder}' not found.")
        return []

def get_xml_file_path(automation_folder):
    cfg_folder = os.path.join(automation_folder, "cfg")
    for filename in os.listdir(cfg_folder):
        if filename.endswith(".xml"):
            return os.path.join(cfg_folder, filename)
    return None

def update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value):
    if os.path.exists(xml_file):
        try:
            tree = ET.parse(xml_file)
            root = tree.getroot()

            root.find("StudyRun").text = study_run_value
            root.find("SequenceStartDate").text = sequence_start_date_value
            root.find("NewRateFiles").text = new_rate_files_value

            tree.write(xml_file)
            print(f"Successfully updated tags in {xml_file}.")
        except ET.ParseError as e:
            print(f"Error parsing XML file: {e}")
        except Exception as e:
            print(f"Error updating XML file: {e}")
    else:
        print(f"Error: XML file '{xml_file}' not found.")

def display_xml_content(xml_file):
    if xml_file:
        try:
            with open(xml_file, 'r') as f:
                xml_data = f.read()
                text_area.delete("1.0", tk.END)
                text_area.insert(tk.INSERT, xml_data)
                print(f"XML content of {xml_file} displayed in the UI.")
        except Exception as e:
            print(f"Error reading XML file: {e}")
            text_area.delete("1.0", tk.END)
            text_area.insert(tk.INSERT, "Error: Could not display XML content.")
    else:
        print("No XML file found for the selected sequence.")
        text_area.delete("1.0", tk.END)
        text_area.insert(tk.INSERT, "No XML file found.")

def combobox_selected(event):
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = os.path.join("AutomationSequences", selected_sequence)
        xml_file = get_xml_file_path(automation_folder)
        display_xml_content(xml_file)

def save_button_clicked():
    selected_sequence = combobox.get()
    if selected_sequence:
        automation_folder = os.path.join("AutomationSequences", selected_sequence)
        xml_file = get_xml_file_path(automation_folder)

        study_run_value = entry_study_run.get()
        sequence_start_date_value = entry_sequence_start_date.get()
        new_rate_files_value = entry_new_rate_files.get()

        update_xml_tags(xml_file, study_run_value, sequence_start_date_value, new_rate_files_value)
        display_xml_content(xml_file)

def save_xml_file(xml_file, xml_data):
    try:
        with open(xml_file, 'w') as f:
            f.write(xml_data)
        print(f"XML file saved successfully: {xml_file}")
    except Exception as e:
        print(f"Error saving XML file: {e}")

def run_powershell_script():
    script_path = os.path.join("SPIA", "Getusername.ps1")
    try:
        result = subprocess.run(["powershell", "-File", script_path], capture_output=True, text=True, check=True)
        print("PowerShell script executed successfully.")
        print("Output:")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error executing PowerShell script: {e}")
        print("Error Output:")
        print(e.stderr)

root = tk.Tk()
root.title("Measure - Experience Study Processing")
root.geometry('700x500')

lbl_Header = tk.Label(root, text="Experience Study Processing", font=('Comic Sans MS', 15, 'bold')).pack(pady=10)
lbl_confg = tk.Label(root, text="Select Measure Automation Sequence", font=('Verdana', 10, 'bold')).pack()

automation_sequences = get_automation_folders("AutomationSequences")
combobox = ttk.Combobox(root, values=automation_sequences, width=35)
combobox.pack()
combobox.bind("<<ComboboxSelected>>", combobox_selected)

text_area = tk.Text(root, width=70, height=20)
text_area.pack(padx=10, pady=10)

# Labels for text entry fields
lbl_study_run = tk.Label(root, text="StudyRun:")
lbl_study_run.pack()
entry_study_run = tk.Entry(root)
entry_study_run.pack()

lbl_sequence_start_date = tk.Label(root, text="SequenceStartDate:")
lbl_sequence_start_date.pack()
entry_sequence_start_date = tk.Entry(root)
entry_sequence_start_date.pack()

lbl_new_rate_files = tk.Label(root, text="NewRateFiles:")
lbl_new_rate_files.pack()
entry_new_rate_files = tk.Entry(root)
entry_new_rate_files.pack()

# Save button
save_button = tk.Button(root, text="Save XML", command=save_button_clicked)
save_button.pack()

# Button to trigger PowerShell script
run_script_button = tk.Button(root, text="Run PowerShell Script", command=run_powershell_script)
run_script_button.pack()

root.mainloop()
